---
## Importing Required Libraries

In [1]:
!pip install roboflow
!pip install ultralytics

from google.colab import userdata
from ultralytics import YOLO
import matplotlib.pyplot as plt
from roboflow import Roboflow
import os
import yaml
import shutil

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 105.2 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.13.0.92
    Uninstalling opencv-python-headless-4.13.0.92:
      Successfully uninstalled opencv-python-headless-4.13.0.92
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.1 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with

##Setting Roboflow API

In [2]:
Roboflow_api_key=userdata.get('roboflow_api_key')

---
## Downloading Dataset

In [3]:
rf = Roboflow(api_key=Roboflow_api_key)
project = rf.workspace("imagerecognition-43zpb").project("helmet-detection-ntbfz")
version = project.version(18)
dataset = version.download("yolo26")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to helmet-detection-18 in yolo26:: 100%|██████████| 25739/25739 [00:04<00:00, 6403.60it/s] 


##Configuring Yaml

In [4]:
# -----------------------------------
# DATA PATH
# -----------------------------------
DATASET_ROOT = "/content/helmet-detection-18"
DATA_YAML_PATH = os.path.join(DATASET_ROOT, "data.yaml")

In [5]:
with open('/content/helmet-detection-18/data.yaml', 'r') as f:
    print(f.readlines())

['train: ../train/images\n', 'val: ../valid/images\n', 'test: ../test/images\n', '\n', 'nc: 4\n', "names: ['bicyclist', 'driver', 'helmet', 'no-helmet']\n", '\n', 'roboflow:\n', '  workspace: imagerecognition-43zpb\n', '  project: helmet-detection-ntbfz\n', '  version: 18\n', '  license: CC BY 4.0\n', '  url: https://universe.roboflow.com/imagerecognition-43zpb/helmet-detection-ntbfz/dataset/18']


- we have 4 names i.e., names: ['bicyclist', 'driver', 'helmet', 'no-helmet'].
- we need just helmet and no-helmet

In [6]:
# -----------------------------
# CONFIG
# -----------------------------

LABEL_SPLITS = ["train", "valid", "test"]

# Mapping: old_index → new_index
CLASS_MAPPING = {
    3: 0,   # no-helmet → 0
    2: 1    # helmet → 1
}

# -----------------------------
# FUNCTION TO FILTER + REMAP
# -----------------------------
def filter_label_file(label_path):
    with open(label_path, "r") as f:
        lines = f.readlines()

    new_lines = []

    for line in lines:
        parts = line.strip().split()
        old_class = int(parts[0])

        if old_class in CLASS_MAPPING:
            new_class = CLASS_MAPPING[old_class]
            parts[0] = str(new_class)
            new_lines.append(" ".join(parts))

    # overwrite file
    with open(label_path, "w") as f:
        f.write("\n".join(new_lines))


# -----------------------------
# PROCESS ALL LABEL FILES
# -----------------------------
for split in LABEL_SPLITS:
    label_dir = os.path.join(DATASET_ROOT, split, "labels")

    if not os.path.exists(label_dir):
        continue

    for file in os.listdir(label_dir):
        if file.endswith(".txt"):
            label_path = os.path.join(label_dir, file)
            filter_label_file(label_path)

print("✅ Label filtering and re-indexing completed!")


# -----------------------------
# UPDATE data.yaml
# -----------------------------
yaml_path = os.path.join(DATASET_ROOT, "data.yaml")

with open(yaml_path, "r") as f:
    data_yaml = yaml.safe_load(f)

data_yaml["nc"] = 2
data_yaml["names"] = ["no-helmet", "helmet"]

with open(yaml_path, "w") as f:
    yaml.safe_dump(data_yaml, f)

print("✅ data.yaml updated to:")
print("   0 → no-helmet")
print("   1 → helmet")


✅ Label filtering and re-indexing completed!
✅ data.yaml updated to:
   0 → no-helmet
   1 → helmet


##Model Loading

In [7]:
# -----------------------------------
# LOAD MODEL
# -----------------------------------
model = YOLO("yolo26s.pt")


##Model Training

In [8]:
# -----------------------------------
# TRAIN
# -----------------------------------
results = model.train(
    data=DATA_YAML_PATH,
    epochs=50,
    imgsz=640,
    batch=16,
    optimizer="AdamW",
    lr0=1e-3,
    patience=5,
    amp=True,
    close_mosaic=30,
    project="/content",
    name="helmet_exp",
    exist_ok=True
)

print("✅ Training completed!")


Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.9.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=30, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/helmet-detection-18/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo26s.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=helmet_exp, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=5,

##Save Best Model

In [10]:
# -----------------------------------
# SAVE BEST MODEL SEPARATELY
# -----------------------------------
best_model_path = "/content/helmet_exp/weights/best.pt"
save_path = "/content/best_helmet_model.pt"

if os.path.exists(best_model_path):
    shutil.copy(best_model_path, save_path)
    print(f"✅ Best model saved at: {save_path}")
else:
    print("❌ best.pt not found!")


✅ Best model saved at: /content/best_helmet_model.pt
